In [ ]:
# Please always run this cell at the beginning of your notebook!
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

np.random.seed(42)  # please make sure you run this line so that your results are reproducible

WF_YEAR_FILE = 'modis_2018_United_Kingdom.csv'
WF_RESOLUTION_KM = 1.0  # set resolution to 4 km
WF_RESOLUTION_ARCSEC = 30  # 120 arcsec ~ 4 km

# geographic boundaries for UK. format: (lon_min, lat_min, lon_max, lat_max)
GEO_BOUND_UK = (-10, 49, 2, 61)

In [ ]:
from climada_petals.hazard import WildFire
from climada.hazard import Centroids
from climada.util.constants import ONE_LAT_KM

In [ ]:
# set the hazard resolution to 1 km. The provided data is at 1 km. Need a scaling factor of 1/1:
res_factor = 1./WF_RESOLUTION_KM

In [ ]:
# This function is provided for you.
def select_region(df, bound):
    """
    Select the region of interest from the dataframe.
    df: hazard dataframe
    bound: tuple in format (lon_min, lat_min, lon_max, lat_max)
    """
    return df[(df['longitude'] >= bound[0]) & 
              (df['latitude'] >= bound[1]) & 
              (df['longitude'] <= bound[2]) & 
              (df['latitude'] <= bound[3])]

In [ ]:
### load the year data
df_firms_year = pd.read_csv(os.path.join('../climada_petals/data', WF_YEAR_FILE))
# focus on California region
df_firms_year_uk = select_region(df_firms_year, GEO_BOUND_UK)
df_firms_year_uk

In [ ]:
wf_year_uk = WildFire()
wf_year_uk.set_hist_fire_seasons_FIRMS(df_firms_year_uk, centr_res_factor=res_factor)

### set_proba_fire_seasons default function calling

In [ ]:
### simulate.
n_ignitions = int(wf_year_uk.n_fires.item())
ign_range = [n_ignitions,n_ignitions+1]

In [ ]:
wf_year_uk.ProbaParams.prop_proba = 0.21
# IMPORTANT: set keep_all_fires=True to keep all fires in the simulation.
# This will allow you to access the simulated event later.
wf_year_uk.set_proba_fire_seasons(n_fire_seasons=1, n_ignitions=ign_range, keep_all_fires=True)
print('The probabilistic season is appended to the historic season:', 
      "event_names", wf_year_uk.event_name,
      "event_ids", wf_year_uk.event_id)

In [ ]:
### plot the new simulated event. can be accessed through .event_id
wf_year_uk.plot_intensity(event=wf_year_uk.event_id[-1])

### Generate coordinates of fired points

In [ ]:
def simulate_fire_probability_array_from_ML_algorithms():
    # return an array of fire probabilities and corresponding coordinates
    res_deg = WF_RESOLUTION_ARCSEC / ONE_LAT_KM
    centroids_uk = Centroids.from_pnt_bounds(GEO_BOUND_UK, res_deg)
    centroids_uk.set_meta_to_lat_lon()
    centr_lonlats = list(zip(centroids_uk.lon, centroids_uk.lat))
    # Generate an array of random numbers from a uniform distribution to simulate the outputs of ML algorithms
    fire_probs = np.random.rand(len(centr_lonlats))
    return centr_lonlats, fire_probs

centr_lonlats, fire_probs = simulate_fire_probability_array_from_ML_algorithms()

def generate_coordinates_of_ignited_points(centr_lonlats, fire_probs, n_ignitions):
    # Normalize the array to create a multinomial distribution
    normalized_probs = fire_probs / np.sum(fire_probs)
    # Draw n_ignitions samples from the multinomial distribution based on the probabilities.
    draws = np.random.choice(len(normalized_probs), size=n_ignitions, p=normalized_probs)
    
    ignited_centr_lonlats = [centr_lonlats[idx] for idx in draws]
    return ignited_centr_lonlats

n_ignitions = 10
ignited_centr_lonlats = generate_coordinates_of_ignited_points(centr_lonlats, fire_probs, n_ignitions)
ignited_centr_lonlats
    

### Function calling with selected fired centroids

In [ ]:
# fired_coords = (-6, 50)
# lon, lat, idx = wf_uk.centroids.get_cloest_point(fired_coords[0], fired_coords[1])
# idx

In [ ]:
from climada_petals.hazard.wildfire import WildFireFuture

wf_uk = WildFireFuture()
wf_uk.set_hist_fire_seasons_FIRMS(df_firms_year_uk, centr_res_factor=res_factor)

In [ ]:
fired_coords = ignited_centr_lonlats # the coords of fire center obtained from ML algorithm
# get_cloest_point takes in lon, lat
centr_id_list = [wf_uk.centroids.get_cloest_point(lon, lat)[2] for lon, lat in fired_coords]
centr_id_list

The length of fired_id_list should be equal to n_ignitions

In [ ]:
### simulate
# n_ignitions = int(wf_uk.n_fires.item())
ign_range = [n_ignitions,n_ignitions+1]


In [ ]:
wf_uk.ProbaParams.prop_proba = 0.21
# IMPORTANT: set keep_all_fires=True to keep all fires in the simulation.
# This will allow you to access the simulated event later.
wf_uk.set_proba_fire_seasons(n_fire_seasons=1, n_ignitions=ign_range, centr_id_list=centr_id_list, keep_all_fires=True)
print('The probabilistic season is appended to the historic season:',
        "event_names", wf_uk.event_name,
        "event_ids", wf_uk.event_id)

In [ ]:
wf_uk.plot_intensity(event=wf_uk.event_id[2])